In [1]:
import h5py
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense  
from keras import backend as K

2022-09-16 15:11:16.786986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 15:11:17.774223: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-16 15:11:19.689007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-16 15:11:19.689240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
dataPath = '/home/WindowsDrive/Shared/Top_Tagging_Data/test.h5'
# trainDictFile = open('trainDict.pkl', 'w')
# testDictFile = open('testDict.pkl', 'w')
tagger_type = 'pfn'

In [3]:
def constituentProcessing(data_dict, max_constits):
    pt = data_dict['fjet_clus_pt'][:,:max_constits]
    eta = data_dict['fjet_clus_eta'][:,:max_constits]
    phi = data_dict['fjet_clus_phi'][:,:max_constits]
    energy = data_dict['fjet_clus_E'][:,:max_constits]


In [8]:
with h5py.File(dataPath, 'r') as data:

    dataKeys = data.keys()
    numJets = data.attrs.get('num_jets')
    halfNumJets = int(numJets / 2)

    dataArr = []

    keysToUse = []
    for key in data.attrs.get('constit'):
        keysToUse.append(key)
    for key in ['fjet_eta', 'fjet_m', 'fjet_phi', 'fjet_pt']:
        keysToUse.append(key)

    for key in keysToUse:
        dataArr.append(data[key][...])

    # trainDict = {key: data[key][:halfNumJets,...] for key in data.attrs.get('constit')}
    # testDict = {key: data[key][halfNumJets:,...] for key in data.attrs.get('constit')}

    # pickle.dump(trainDict, trainDictFile)
    # pickle.dump(testDict, testDictFile)

    dataLabels = data['labels']
    dataWeights = data['weights']

    # print(list(dataLabels))

    # print(dataArr[0])


['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt', 'fjet_clus_E', 'fjet_eta', 'fjet_m', 'fjet_phi', 'fjet_pt']


In [10]:
print(dataKeys)

ValueError: Invalid group (or file) id (invalid group (or file) ID)